In [665]:
!pip install igraph 


In [666]:
import igraph
import numpy as np
import numpy as np
from numpy import linalg as la
import time
import re
import dgl
import random
from pandas import DataFrame
import torch
from torch.nn import MSELoss
from torch.nn import functional as F
from IPython.display import display
import networkx as nx
from tqdm import tqdm



In [667]:
def decode_ENAS_to_igraph(row):
    if type(row) == str:
        row = eval(row)  # convert string to list of lists
    n = len(row)
    g = igraph.Graph(directed=True)
    g.add_vertices(n)
    # g.vs[0]['type'] = 0  # input node
    for i, node in enumerate(row):
        g.vs[i]['type'] = node[0]  # assign 2, 3, ... to other types
        g.add_edge(i, i+1)  # always connect from last node
        for j, edge in enumerate(node[1:]):
            if edge == 1:
                g.add_edge(j, i)
    # g.vs[n+1]['type'] = 1  # output node
    # g.add_edge(n, n+1)
    # note that the nodes 0, 1, ... n+1 are in a topological order
    return g, n

In [668]:
import ast

def form_adjacency_matrix(row):
    # Convert the string representation to a list
    nodes = row
    
    # Number of nodes (excluding the first element of each sublist which is the type)
    num_nodes = len(nodes)
    
    # Initialize the adjacency matrix with zeros
    adj_matrix = [[0 for _ in range(num_nodes)] for _ in range(num_nodes)]
    
    # Iterate over each node's connections
    for i, node in enumerate(nodes):
        for j, connected in enumerate(node[1:]):  # Skip the first element (node type)
            if connected == 1:
                # Update the adjacency matrix; node[i] is connected to node[j]
                adj_matrix[j][i] = 1  # Note: j indexes into the sublist indicating connections
    
    return np.array(adj_matrix)


In [669]:
import torch
def one_hot(idx, length):
    idx = torch.LongTensor([idx]).unsqueeze(0)
    x = torch.zeros((1, length)).scatter_(1, idx, 1)
    return x
    

In [670]:
import networkx as nx

In [671]:
def compute_Dq(dag: nx.DiGraph, target_node: str, only_diag: bool = True,
               verbose: bool = False, ordered: bool = False) -> np.ndarray:
    """
    Compute Dq, the frequency response matrix of the GSO associated with node q, based on the
    existence of paths from each node to the target node.

    Args:
        dag (nx.DiGraph): Directed acyclic graph (DAG).
        target_node (str): Target node identifier.
        only_diag (bool, optional): Whether to return only the diagonal of the matrix. Defaults to True.
        verbose (bool, optional): Whether to print verbose output. Defaults to False.

    Returns:
        np.ndarray: Frequency response matrix Dq.
    """
    N = dag.number_of_nodes()
    target_idx = ord(target_node) - ord('a') if isinstance(target_node, str) else target_node
    
    path_exists = np.zeros(N)
    max_node = target_idx + 1 if ordered else N 
    for i in range(max_node):
        path_exists[i] = nx.has_path(dag, i, target_idx)
    
    if verbose:
        for i, exists in enumerate(path_exists):
            print(f'Has path from node {i} to node {target_node}: {exists}')

    if only_diag:
        return path_exists
    else:
        return np.diag(path_exists)

def compute_GSOs(W, dag):
    N = W.shape[0]
    GSOs = np.array([W @ compute_Dq(dag, i, N) @ la.inv(W) for i in range(N)])
    return GSOs

In [674]:
import numpy as np

def form_one_hot_matrix_fixed_types(nodes):
    # Number of types is known to be 6
    num_types = 6
    
    # Initialize the one-hot encoding matrix with zeros
    one_hot_matrix = np.zeros((len(nodes), num_types), dtype=int)
    
    # Populate the one-hot encoding matrix
    for i, node in enumerate(nodes):
        node_type = node[0]
        one_hot_matrix[i, node_type] = 1
    
    return one_hot_matrix


In [675]:
def OneHot(name, n_types=6, with_y=True, burn_in=1000):
    # load ENAS format NNs to igraphs or tensors
    g_list = []
    max_n = 0  # maximum number of nodes
    with open('/Users/hamedajorlou/dag_conv_nn/src/%s.txt' % name, 'r') as f:
        for i, row in enumerate(tqdm(f)):
            if i < burn_in:
                continue
            if row is None:
                break
            if with_y:
                row, y = eval(row)
            else:
                row = eval(row)
            print(row)
            adj = form_one_hot_matrix_fixed_types(row)
            g_list.append(adj)
    return g_list

In [676]:
P = OneHot('final_structures6', n_types=6, with_y=True, burn_in=1000)

8942it [00:00, 41597.18it/s]

[[1], [0, 0], [5, 1, 1], [4, 1, 1, 1], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
[[2], [4, 0], [0, 0, 1], [3, 1, 0, 0], [0, 0, 0, 0, 0], [5, 0, 0, 0, 0, 0]]
[[3], [2, 0], [3, 1, 1], [5, 1, 0, 0], [0, 0, 1, 0, 1], [2, 0, 0, 0, 0, 0]]
[[0], [0, 0], [2, 0, 0], [3, 0, 0, 0], [3, 1, 0, 0, 1], [3, 0, 1, 0, 1, 0]]
[[3], [3, 0], [3, 1, 0], [3, 0, 0, 0], [3, 0, 0, 0, 0], [4, 0, 1, 0, 0, 0]]
[[4], [0, 0], [3, 0, 1], [1, 0, 0, 0], [4, 1, 0, 1, 0], [5, 0, 1, 0, 0, 0]]
[[2], [3, 0], [3, 1, 0], [2, 0, 0, 0], [2, 0, 0, 1, 0], [0, 1, 0, 0, 1, 0]]
[[0], [2, 0], [0, 0, 0], [3, 0, 0, 1], [4, 0, 1, 0, 0], [1, 0, 0, 0, 1, 0]]
[[0], [3, 1], [3, 0, 0], [5, 0, 0, 0], [4, 0, 1, 0, 0], [2, 0, 0, 0, 0, 0]]
[[5], [5, 0], [4, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0, 0], [4, 0, 0, 1, 1, 0]]
[[4], [1, 0], [5, 1, 0], [4, 0, 0, 0], [2, 0, 0, 0, 0], [5, 1, 0, 0, 1, 0]]
[[4], [2, 1], [1, 0, 0], [0, 0, 0, 0], [0, 1, 0, 1, 1], [2, 0, 0, 1, 1, 0]]
[[2], [4, 1], [3, 1, 0], [1, 0, 0, 0], [3, 1, 0, 0, 0], [2, 1, 1, 0, 1, 1]]
[[5], [0, 1]

17139it [00:00, 39230.31it/s]

[[3], [2, 0], [4, 1, 0], [1, 1, 0, 0], [3, 0, 1, 0, 0], [3, 0, 0, 0, 0, 0]]
[[3], [0, 0], [0, 0, 1], [5, 1, 0, 0], [0, 0, 1, 1, 0], [1, 0, 0, 0, 0, 0]]
[[5], [2, 0], [2, 1, 0], [3, 0, 0, 0], [5, 1, 1, 0, 1], [2, 0, 1, 0, 0, 0]]
[[1], [5, 1], [1, 0, 1], [4, 0, 0, 0], [2, 0, 0, 0, 1], [3, 1, 1, 0, 0, 0]]
[[0], [2, 1], [3, 0, 0], [3, 0, 1, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0]]
[[5], [2, 0], [0, 1, 0], [2, 0, 0, 1], [4, 0, 1, 0, 0], [4, 0, 0, 0, 0, 0]]
[[2], [0, 0], [2, 0, 0], [4, 1, 0, 1], [1, 0, 1, 1, 0], [0, 0, 0, 0, 0, 1]]
[[0], [0, 1], [5, 0, 0], [0, 0, 1, 0], [3, 1, 0, 1, 0], [4, 0, 0, 0, 0, 0]]
[[5], [5, 0], [4, 0, 0], [4, 0, 0, 0], [4, 0, 0, 0, 1], [3, 0, 1, 0, 0, 1]]
[[0], [3, 0], [4, 1, 0], [5, 0, 0, 0], [1, 1, 0, 0, 1], [3, 1, 1, 1, 1, 1]]
[[5], [3, 0], [2, 1, 0], [3, 0, 1, 0], [5, 0, 0, 0, 0], [0, 0, 1, 1, 0, 0]]
[[3], [5, 0], [0, 0, 0], [4, 1, 0, 1], [5, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0]]
[[1], [5, 0], [0, 0, 0], [1, 1, 0, 0], [4, 1, 1, 1, 0], [1, 0, 0, 0, 1, 0]]
[[4], [5, 1]

20020it [00:00, 37684.84it/s]

[[5], [3, 1], [2, 0, 0], [4, 0, 0, 0], [0, 0, 0, 0, 0], [5, 0, 0, 0, 0, 0]]
[[5], [5, 0], [5, 1, 0], [5, 0, 1, 1], [3, 1, 0, 1, 1], [3, 1, 0, 1, 0, 0]]
[[5], [2, 0], [1, 1, 0], [2, 0, 1, 0], [0, 1, 1, 1, 0], [0, 1, 0, 0, 0, 0]]
[[1], [1, 0], [4, 0, 1], [3, 0, 1, 0], [4, 0, 0, 1, 0], [4, 0, 1, 0, 1, 1]]
[[4], [4, 1], [4, 0, 0], [4, 1, 0, 1], [4, 0, 0, 1, 0], [3, 0, 0, 0, 0, 1]]
[[0], [5, 0], [0, 0, 0], [2, 1, 0, 0], [1, 1, 0, 0, 1], [5, 0, 0, 1, 0, 0]]
[[0], [5, 0], [2, 0, 0], [5, 0, 1, 0], [4, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0]]
[[0], [0, 0], [4, 0, 0], [2, 0, 0, 0], [3, 1, 0, 0, 0], [1, 0, 0, 1, 1, 0]]
[[2], [5, 1], [5, 1, 0], [4, 0, 1, 1], [4, 0, 0, 1, 0], [4, 1, 1, 0, 0, 1]]
[[4], [0, 1], [5, 0, 0], [3, 0, 0, 0], [0, 1, 1, 1, 1], [4, 1, 0, 0, 1, 1]]
[[1], [3, 0], [1, 0, 1], [0, 0, 0, 0], [5, 0, 1, 0, 1], [1, 0, 0, 0, 0, 0]]
[[3], [4, 0], [5, 1, 1], [0, 0, 0, 0], [0, 1, 0, 1, 1], [5, 1, 0, 0, 0, 0]]
[[1], [2, 1], [2, 1, 0], [3, 0, 1, 0], [5, 0, 1, 0, 0], [0, 0, 1, 1, 0, 1]]
[[0], [2, 0]

In [677]:
X_train = np.stack(P,axis=0)


In [678]:
dag = nx.from_numpy_array(Q[0].T, create_using=nx.DiGraph())

In [679]:
def Y_trainFunction(name, n_types=6, with_y=True, burn_in=1000):
    # load ENAS format NNs to igraphs or tensors
    y_list = []
    max_n = 0  # maximum number of nodes
    with open('/Users/hamedajorlou/dag_conv_nn/src/%s.txt' % name, 'r') as f:
        for i, row in enumerate(tqdm(f)):
            if i < burn_in:
                continue
            if row is None:
                break
            if with_y:
                row, y = eval(row)
            else:
                row = eval(row)
            y_list.append(y)
            # adj = form_adjacency_matrix(row)
            # g_list.append(adj)
    return y_list

In [680]:
Y = Y_trainFunction('final_structures6', n_types=6, with_y=True, burn_in=1000)
Y_train = np.array(Y)

0it [00:00, ?it/s]

20020it [00:00, 45511.26it/s]


In [681]:
def ADJ(name, n_types=6, with_y=True, burn_in=1000):
    # load ENAS format NNs to igraphs or tensors
    g_list = []
    max_n = 0  # maximum number of nodes
    with open('/Users/hamedajorlou/dag_conv_nn/src/%s.txt' % name, 'r') as f:
        for i, row in enumerate(tqdm(f)):
            if i < burn_in:
                continue
            if row is None:
                break
            if with_y:
                row, y = eval(row)
            else:
                row = eval(row)
            # print(len(row))
            adj = form_adjacency_matrix(row)
            g_list.append(adj)
    return g_list

In [682]:
Q = ADJ('final_structures6', n_types=6, with_y=True, burn_in=1000)

20020it [00:00, 37676.13it/s]


In [683]:
IO1 = []
for i in range(len(Q)):
    dag = nx.from_numpy_array(Q[i].T, create_using=nx.DiGraph())
    W = la.inv(np.eye(Q[i].shape[0]) - Q[i])
    W_inf = la.inv(W)
    GSOs1 = np.array([(W * compute_Dq(dag, i, Q[i].shape[0])) @ W_inf for i in range(Q[i].shape[0])])
    IO1.append(GSOs1)

In [684]:
GSOs = np.stack(IO1, axis=0)  # Stacks along the new axis


In [686]:
X_train.shape

(19020, 6, 6)

In [687]:
Y_train.shape

(19020,)

In [688]:
GSOs.shape

(19020, 6, 6, 6)